<a href="https://colab.research.google.com/github/Hamza-Chekireb/llama70b-openai-api-Q-A/blob/main/Llama3_70b_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.Import Requirements

In [ ]:
! pip install openai

In [ ]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install langchain
!pip install langchain_community
!pip install chromadb


In [ ]:
from IPython.display import display, Markdown
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain import HuggingFacePipeline
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline # AutoModelForCausalLM is used to add layer for application like QA
import langchain
from langchain import HuggingFacePipeline
from openai import OpenAI

### 2.Creat the Vectore Data Base

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
DATA_PATH = '/content/'
DB_FAISS_PATH = 'vectorstore/db_chroma'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    # embeddings = HuggingFaceInstructEmbeddings(
    # model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
# )

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})

    # 4. Creat Vector Data Base
    db = Chroma.from_documents(texts,embeddings,persist_directory=DB_FAISS_PATH)
    # store the db in repertory
    db.persist()

if __name__ == "__main__":
    create_vector_db()

### 4.Output Function

In [ ]:
# Create new connection to VDB

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': DEVICE})
VDBPath = 'vectorstore/db_chroma'
vdb_connection = Chroma(persist_directory=VDBPath,  embedding_function=embeddings)


In [ ]:
question = ""
context = ""
promp_template_for_processing = f"Answer this Question ({question}) in an exhaustive and concise way, based only on the given context.\n\n" \
+ f"Context: {context}\n\n" + f"Question : {question}\n"

In [ ]:
from langchain import OpenAI, LLMChain, ConversationBufferMemory

# Initialize the OpenAI client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="nvapi-tlopcFnGdWAxQJ9wgoIG8cNJ5XGUO1dpYcvLyULNNdQXGUNxJ5b"
)

# Define the model
model_name = "meta/llama3-70b-instruct"

# Initialize memory component
memory = ConversationBufferMemory()

# Create the LLM Chain with memory
llm_chain = LLMChain(
    llm=client,
    prompt_template="You are an AI assistant. Continue the conversation below:\n\n{history}\n\nUser: {user_input}\nAssistant:",
    memory=memory
)

def generate_response(user_input, chain):
    response = chain.run(user_input)
    return response

# Example usage
user_input = "Write a limerick about the wonders of GPU computing."
response = generate_response(user_input, llm_chain)
print("Model:", response)

# Further interaction example
user_input = "Tell me more about the history of GPU computing."
response = generate_response(user_input, llm_chain)
print("Model:", response)


In [ ]:
#output function
def final_result(question):
  docs = vdb_connection.similarity_search(question,k=3)
  context = "\n\n ".join([doc.page_content for doc in docs])

  promp_template_for_processing = f"Answer this Question ({question}) in an exhaustive and concise way, based only on the given context.\n\n" \
+ f"Context: {context}\n\n" + f"Question : {question}\n"

  client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

  completion = client.chat.completions.create(
    model="meta/llama3-70b-instruct",
    messages=[{"role":"user","content":promp_template_for_processing}],
    temperature=0.1,
    top_p=1,
    max_tokens=1024,
    stream=True
  )

  response = ""
  for chunk in completion:
      if chunk.choices[0].delta.content is not None:
          response += chunk.choices[0].delta.content
  return display(Markdown(response))


In [ ]:
while True:
    question = input("Ask me a question? (Type 'exit' to end): ")
    if question.lower() == "exit":
        break
    result = final_result(question)
    result